# VacationPy
----

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
import warnings
warnings.filterwarnings("ignore")

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weatherDF = pd.read_csv("outputData/WeatherData.csv")
weatherDF.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Atuona,-9.8000,-139.0333,76.93,77,3,15.43,PF,1658836722
1,Lompoc,34.6391,-120.4579,58.01,88,100,5.01,US,1658836723
2,Shiyan,32.6475,110.7781,77.41,91,30,4.50,CN,1658836723
3,Mamou,10.5000,-12.0000,74.52,84,98,2.46,GN,1658836723
4,Bandarbeyla,9.4942,50.8122,90.23,43,81,26.37,SO,1658836724


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

# create coordinates by joining lat and long
locations = weatherDF[["Lat", "Lng"]]

# Humidity as the weights
humidity = weatherDF["Humidity"].astype(float)

# Plotting the heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=1.3)

# Create heat layer
heatLayer = gmaps.heatmap_layer(locations, weights=humidity, max_intensity = 100)

# Add layer
fig.add_layer(heatLayer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [13]:
idealWeatherDF = weatherDF.loc[(weatherDF["Max Temp"] < 80) & (weatherDF["Max Temp"] > 70) & (weatherDF["Wind Speed"] < 10) & (weatherDF["Cloudiness"] == 0) & (weatherDF["Humidity"] < 80)]
len(idealWeatherDF)

8

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotelDF = idealWeatherDF
hotelDF["Hotel Name"] = ""
hotelDF.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
12,Port Elizabeth,-33.9180,25.5701,77.09,50,0,9.22,ZA,1658836726,
32,Cape Town,-33.9258,18.4232,74.12,44,0,2.30,ZA,1658836733,
155,Bonito,-21.1211,-56.4819,74.70,38,0,2.86,BR,1658836777,
179,Kruisfontein,-34.0033,24.7314,74.88,24,0,8.01,ZA,1658836785,
188,Portland,45.5234,-122.6762,72.16,77,0,1.54,US,1658836760,


In [6]:
# search for hotels nearby
# dictionary of parameters to search for hotels nearby
parameters = {
    "type": "lodging",
    "radius": 5000,
    "key": g_key
}

# Loop through the hotel dataframe and run a lat/long search
for index, row in hotelDF.iterrows():

    # create lat and long variable
    lat = row["Lat"]
    lng = row["Lng"]

    # Update parameters dictionary
    parameters["location"] = f"{lat},{lng}"
    
    # Define baseURL
    baseURL = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make API request
    response = requests.get(baseURL, params=parameters).json()
    
    # Add data to DF
    try:
        hotelDF.loc[index,"Hotel Name"] = response["results"][0]["name"]
    
    except:
        print(f"Unable to find hotel near {hotelDF.loc[index, 'City']}")

In [11]:
hotelDF

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
12,Port Elizabeth,-33.9180,25.5701,77.09,50,0,9.22,ZA,1658836726,39 On Nile Guest House
32,Cape Town,-33.9258,18.4232,74.12,44,0,2.30,ZA,1658836733,"Radisson Blu Hotel Waterfront, Cape Town"
155,Bonito,-21.1211,-56.4819,74.70,38,0,2.86,BR,1658836777,Marruá Hotel
179,Kruisfontein,-34.0033,24.7314,74.88,24,0,8.01,ZA,1658836785,Oyster Bay House Rental
188,Portland,45.5234,-122.6762,72.16,77,0,1.54,US,1658836760,Embassy Suites by Hilton Portland Downtown
429,Cassilândia,-19.1133,-51.7342,74.97,38,0,4.54,BR,1658836841,Delirius Motel
466,Oudtshoorn,-33.5907,22.2014,70.16,23,0,2.91,ZA,1658836577,Earthbound Guest House
524,Union,40.6334,-74.2663,76.15,69,0,5.75,US,1658836563,Homewood Suites by Hilton Newark-Cranford


In [8]:
# adding a clickable tool tip to the marker
infoBox = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
hotelInfo = [infoBox.format(**row) for index, row in hotelDF.iterrows()]

# Store the locations
coordinates = hotelDF[["Lat", "Lng"]]

# Add a marker layer on the heatmap 
markerLayer = gmaps.marker_layer(coordinates, info_box_content=hotelInfo)

# Add marker layers
fig.add_layer(markerLayer)

In [12]:
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))